## 1. Environment Setup

`(1) Env Environment Variables`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) Basic Libraries`

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

## 2. StateGraph
- A graph structure that operates based on state.
- Practice: Restaurant Menu Recommendation System
    - Recommends menus based on user preferences and provides information about the menu.

`(1) State`
- The state defines the structure of the data processed by the graph.
- Overrides the existing state.

In [3]:
from typing import TypedDict

# Define State Schema - stores user preference, recommended menu, and menu information
class MenuState(TypedDict):
    user_preference: str
    recommended_menu: str
    menu_info: str

`(2) Node`
- A node is a function that performs the actual work in the graph.

In [4]:
import random

def get_user_preference(state: MenuState) -> MenuState:
    print("---Generating Random User Preference---")
    preferences = ["Meat", "Seafood", "Vegetarian", "Anything"]
    preference = random.choice(preferences)
    print(f"Generated Preference: {preference}")
    return {"user_preference": preference}

def recommend_menu(state: MenuState) -> MenuState:
    print("---Recommending Menu---")
    preference = state['user_preference']
    if preference == "Meat":
        menu = "Steak"
    elif preference == "Seafood":
        menu = "Lobster Pasta"
    elif preference == "Vegetarian":
        menu = "Green Salad"
    else:
        menu = "Chef's Special of the Day"
    print(f"Recommended Menu: {menu}")
    return {"recommended_menu": menu}

def provide_menu_info(state: MenuState) -> MenuState:
    print("---Providing Menu Information---")
    menu = state['recommended_menu']
    if menu == "Steak":
        info = "A juicy steak made with the finest beef. Price: 30,000 KRW"
    elif menu == "Lobster Pasta":
        info = "A harmony of fresh lobster and al dente pasta. Price: 28,000 KRW"
    elif menu == "Green Salad":
        info = "A healthy salad made with fresh organic vegetables. Price: 15,000 KRW"
    else:
        info = "A special dish carefully selected by the chef daily. Price: 35,000 KRW"
    print(f"Menu Information: {info}")
    return {"menu_info": info}

`(3) Graph Construction`
- Build the entire graph using the defined components.

In [5]:
from langgraph.graph import StateGraph, START, END

# Create Graph Builder
builder = StateGraph(MenuState)

# Add Nodes
builder.add_node("get_preference", get_user_preference)
builder.add_node("recommend", recommend_menu)
builder.add_node("provide_info", provide_menu_info)

# Add Edges
builder.add_edge(START, "get_preference")
builder.add_edge("get_preference", "recommend")
builder.add_edge("recommend", "provide_info")
builder.add_edge("provide_info", END)

# Compile Graph
graph = builder.compile()

In [7]:
from IPython.display import Image, display

# Visualize Graph
display(Image(graph.get_graph().draw_mermaid_png()))

In [8]:
# Execute Graph

def print_result(result: MenuState):
    print("\n=== Result ===")
    print("Preference:", result['user_preference'])
    print("Recommended Menu:", result['recommended_menu'])
    print("Menu Information:", result['menu_info'])
    print("============\n")


# Initial Input
inputs = {"user_preference": ""}

# Test by running multiple times
for _ in range(2):
    result = graph.invoke(inputs)
    print_result(result)
    print("*"*100)
    print()

---랜덤 사용자 선호도 생성---
생성된 선호도: 아무거나
---메뉴 추천---
추천 메뉴: 오늘의 쉐프 특선
---메뉴 정보 제공---
메뉴 정보: 쉐프가 그날그날 엄선한 특별 요리입니다. 가격: 35,000원

=== 결과 ===
선호도: 아무거나
추천 메뉴: 오늘의 쉐프 특선
메뉴 정보: 쉐프가 그날그날 엄선한 특별 요리입니다. 가격: 35,000원

****************************************************************************************************

---랜덤 사용자 선호도 생성---
생성된 선호도: 육류
---메뉴 추천---
추천 메뉴: 스테이크
---메뉴 정보 제공---
메뉴 정보: 최상급 소고기로 만든 juicy한 스��이크입니다. 가격: 30,000원

=== 결과 ===
선호도: 육류
추천 메뉴: 스테이크
메뉴 정보: 최상급 소고기로 만든 juicy한 스테이크입니다. 가격: 30,000원

****************************************************************************************************



## 3. Conditional Edge
- Edges define the connections between nodes.
- Conditional Edge: Proceeds to a different path depending on whether the user input is menu-related.

`(1) State Definition`
- If the user input is a menu recommendation, search the vector store and execute the RAG Chain.
- Otherwise, the LLM generates a response.

In [9]:
from typing import List

# State Schema
class MenuState(TypedDict):
    user_query: str
    is_menu_related: bool
    search_results: List[str]
    final_answer: str

`(2) Vector Store Search Tool`
- Initialize the vector store for menu search (load existing store).

In [10]:
from langchain_chroma import Chroma
from langchain_ollama  import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3") 

# Load Chroma Index
vector_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

`(3) Node`

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM Model
llm = ChatOpenAI(model="gpt-4o-mini")

def get_user_query(state: MenuState) -> MenuState:
    user_query = input("How can I help you? ")
    return {"user_query": user_query}

def analyze_input(state: MenuState) -> MenuState:
    analyze_template = """
    Analyze the user's input to determine if it is a question about a restaurant menu recommendation or food information.

    User Input: {user_query}

    Answer "True" if it is a question about a restaurant menu or food information, otherwise answer "False".

    Answer:
    """
    analyze_prompt = ChatPromptTemplate.from_template(analyze_template)
    analyze_chain = analyze_prompt | llm | StrOutputParser()
    
    result = analyze_chain.invoke({"user_query": state['user_query']})
    is_menu_related = result.strip().lower() == "true"
    
    return {"is_menu_related": is_menu_related}

def search_menu_info(state: MenuState) -> MenuState:
    # Search for up to 2 documents in the vector store
    results = vector_db.similarity_search(state['user_query'], k=2)
    search_results = [doc.page_content for doc in results]
    return {"search_results": search_results}

def generate_menu_response(state: MenuState) -> MenuState:
    response_template = """
    User Input: {user_query}
    Menu-related Search Results: {search_results}

    Based on the information above, generate a detailed answer to the user's menu-related question.
    Use the information from the search results to provide accurate and useful information.

    Answer:
    """
    response_prompt = ChatPromptTemplate.from_template(response_template)
    response_chain = response_prompt | llm | StrOutputParser()
    
    final_answer = response_chain.invoke({"user_query": state['user_query'], "search_results": state['search_results']})
    print(f"\nMenu Assistant: {final_answer}")
    
    return {"final_answer": final_answer}

def generate_general_response(state: MenuState) -> MenuState:
    response_template = """
    User Input: {user_query}

    The above input is not related to a restaurant menu or food.
    Generate an appropriate response in a general conversational context.

    Answer:
    """
    response_prompt = ChatPromptTemplate.from_template(response_template)
    response_chain = response_prompt | llm | StrOutputParser()
    
    final_answer = response_chain.invoke({"user_query": state['user_query']})
    print(f"\nGeneral Assistant: {final_answer}")
    
    return {"final_answer": final_answer}

`(4) Edge`

In [12]:
from typing import Literal

def decide_next_step(state: MenuState) -> Literal["search_menu_info", "generate_general_response"]:
    if state['is_menu_related']:
        return "search_menu_info"  
    else:
        return "generate_general_response"

`(5) Graph Construction`

In [13]:
from langgraph.graph import StateGraph, START, END

# Graph Construction
builder = StateGraph(MenuState)

# Add Nodes
builder.add_node("get_user_query", get_user_query)
builder.add_node("analyze_input", analyze_input)
builder.add_node("search_menu_info", search_menu_info)
builder.add_node("generate_menu_response", generate_menu_response)
builder.add_node("generate_general_response", generate_general_response)

# Add Edges
builder.add_edge(START, "get_user_query")
builder.add_edge("get_user_query", "analyze_input")

# Add Conditional Edges
builder.add_conditional_edges(
    "analyze_input",
    decide_next_step,
    {
        "search_menu_info": "search_menu_info",
        "generate_general_response": "generate_general_response"
    }
)

builder.add_edge("search_menu_info", "generate_menu_response")
builder.add_edge("generate_menu_response", END)
builder.add_edge("generate_general_response", END)

# Compile Graph
graph = builder.compile()

In [14]:
from IPython.display import Image, display

# Visualize Graph
display(Image(graph.get_graph().draw_mermaid_png()))

`(6) Execute Graph`

In [15]:
while True:
    initial_state = {'user_query':''}
    graph.invoke(initial_state) 
    continue_chat = input("Do you have any other questions? (y/n): ").lower()
    if continue_chat != 'y':
        print("Ending the conversation. Thank you!")
        break


메뉴 어시스턴트: 스테이크 메뉴�� 가격은 다음과 같습니다:

1. **시그니처 스테이크**  
   - **가격**: ₩35,000  
   - **주요 식재료**: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스  
   - **설명**: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용하며 미디엄 레어로 조리하여 육즙을 최대한 보존합니다. 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여지며, 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.

2. **안심 스테이크 샐러드**  
   - **가격**: ₩26,000  
   - **주요 식재료**: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈  
   - **설명**: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.

이 외에 궁금한 점이 있으면 언제든지 말씀해 주세요!

일반 어시스턴트: 미국의 수도는 워��턴 D.C.입니다.
대화를 종료합니다. 감사합니다!
